In [11]:
from CNN_model import CNNClassifier,Trainer
import torch

device = "cuda" if torch.cuda.is_available() else "cpu"
print(device)

model = CNNClassifier(num_classes=5)
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

cuda


In [12]:
from torchvision import transforms
from torch.utils.data import random_split
from torch.utils.data import Dataset
from PIL import Image

class ImageCSVDataset(Dataset):
    def __init__(self, file_list, transform=None):
        """
        file_list: path to .txt or .csv file
        transform: torchvision transforms
        """
        self.samples = []
        self.transform = transform

        with open(file_list, "r") as f:
            for line in f:
                path, label = line.strip().split(",")
                label = int(label) - 1
                self.samples.append((path, int(label)))

    def __len__(self):
        return len(self.samples)

    def __getitem__(self, idx):
        img_path, label = self.samples[idx]

        image = Image.open(img_path).convert("RGB")

        if self.transform:
            image = self.transform(image)

        return image, label

train_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(
        mean=[0.485, 0.456, 0.406],
        std=[0.229, 0.224, 0.225]
    )
])

val_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(
        mean=[0.485, 0.456, 0.406],
        std=[0.229, 0.224, 0.225]
    )
])

dataset = ImageCSVDataset("data_list.txt", transform=train_transform)

train_size = int(0.8 * len(dataset))
val_size = len(dataset) - train_size

train_dataset, val_dataset = random_split(dataset, [train_size, val_size])

val_dataset.dataset.transform = val_transform

In [13]:
from torch.utils.data import DataLoader

train_loader = DataLoader(
    train_dataset,
    batch_size=32,
    shuffle=True,
)

val_loader = DataLoader(
    val_dataset,
    batch_size=32,
    shuffle=False,
)


In [14]:
trainer = Trainer(
    model=model,
    train_loader=train_loader,
    val_loader=val_loader,
    criterion=criterion,
    optimizer=optimizer,
    device=device,
    output_path="../../weights/CNN/outputs"
)

In [17]:
trainer.fit(epochs=100)

Epoch [1/100] | Train Loss: 1.0382, Train Acc: 0.6296 | Val Loss: 1.0358, Val Acc: 0.6519
Epoch [2/100] | Train Loss: 1.0294, Train Acc: 0.6250 | Val Loss: 1.0339, Val Acc: 0.5519
Epoch [3/100] | Train Loss: 1.0121, Train Acc: 0.6250 | Val Loss: 1.8195, Val Acc: 0.2593
Epoch [4/100] | Train Loss: 1.0522, Train Acc: 0.5722 | Val Loss: 1.4674, Val Acc: 0.2222
Epoch [5/100] | Train Loss: 1.0477, Train Acc: 0.5787 | Val Loss: 1.2186, Val Acc: 0.3852
Epoch [6/100] | Train Loss: 0.9769, Train Acc: 0.6463 | Val Loss: 1.0049, Val Acc: 0.5519
Epoch [7/100] | Train Loss: 0.9888, Train Acc: 0.6194 | Val Loss: 1.1743, Val Acc: 0.3815
Epoch [8/100] | Train Loss: 0.9757, Train Acc: 0.6231 | Val Loss: 1.2469, Val Acc: 0.3519
Epoch [9/100] | Train Loss: 0.9860, Train Acc: 0.6074 | Val Loss: 1.1929, Val Acc: 0.3852
Epoch [10/100] | Train Loss: 0.9254, Train Acc: 0.6759 | Val Loss: 0.9203, Val Acc: 0.7333
Epoch [11/100] | Train Loss: 0.8807, Train Acc: 0.7250 | Val Loss: 0.8811, Val Acc: 0.7222
Epoch [1